In [116]:
import numpy as np

In [117]:
M = 20  # kg
g = 9.8 # m / s^2
m = 0.5 # kg
l = 0.5 # m

In [118]:
v1 = ((M + m) * g) / (M * l)
v2 = -(m * g) / M

c1 = -1 / (M * l)
c2 = 1 / M

In [119]:
A = np.array([[0, 1, 0, 0], [0, 0, v2, 0], [0, 0, 0, 1], [0, 0, v1, 0]])
B = np.array([[0], [c2], [0], [c1]])

In [120]:
print(np.shape(A))
print(np.shape(B))

(4, 4)
(4, 1)


## Controllability

CM = [B AB A<sup>2</sup>B A<sup>3</sup>B]

In [121]:
CM = np.array([[B], [A @ B], [A @ A @ B], [A @ A @ A @ B]]).reshape(4, 4)

In [122]:
print((CM))

[[ 0.      0.05    0.     -0.1   ]
 [ 0.05    0.     -0.1     0.    ]
 [ 0.      0.0245  0.     -2.009 ]
 [ 0.0245  0.     -2.009   0.    ]]


In [123]:
rank = np.linalg.matrix_rank(CM)
print("Rank of CM: ", rank)

Rank of CM:  4


> Rank of CM = 4, therefore controllable

## Stability


In [124]:
print(A)

[[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]


In [125]:
eigenvalues = np.linalg.eigvals(A)

print("Eigenvalues of matrix A:", eigenvalues)

Eigenvalues of matrix A: [ 0.          0.          4.48218697 -4.48218697]


>One eigen value is positive, so the system is unstable

## Policy Iteration


In [134]:
Q = np.array([[-100, 0, 0, 0], [0, -100, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
R = np.array([[0.01]])


In [135]:
print("A", A)
print("B", B)

A [[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]
B [[ 0.  ]
 [ 0.05]
 [ 0.  ]
 [-0.1 ]]


In [136]:
F = [0, 0, 0, 0]
F = np.array(F).reshape(1, 4)
# print eigenvalues of A + BF
print("Eigenvalues of A + BF:", np.linalg.eigvals(A + B @ F))

Eigenvalues of A + BF: [ 0.          0.          4.48218697 -4.48218697]


In [142]:
def policy_iteration(A, B, Q, R, F):
    P = np.ones((4, 4))
    
    
    for i in range(10000):
        for i in range(100):
            P_new = Q + F.T @ R @ F + (A + B @ F).T @ P @ (A + B @ F)
            if np.allclose(P, P_new):
                break
            P = P_new

        F = -np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

    return F

 
F = policy_iteration(A, B, Q, R, F)

print("F", F)

F [[ 0.00000000e+00  0.00000000e+00  2.00389583e+02 -2.36472947e-12]]


In [143]:
import cv2

class InvertedPendulum:
    def __init__(self):
        f = 0

    def step( self, state_vec, t=None ):
        """ state vector :
                x0 : position of the cart
                x1 : veclocity of the cart
                x2 : angle of pendulum. In ref frame with x as forward of the cart and y as up. Angile with respect to ground plane
                x3 : angular velocity of the pendulum
        """
        CART_POS = state_vec[0]
        BOB_ANG  = state_vec[2]*180. / np.pi # degrees
        LENGTH_OF_PENDULUM = 110.

        IM = np.zeros( (512, 512,3), dtype='uint8' )

        # Ground line
        cv2.line(IM, (0, 450), (IM.shape[1], 450), (19,69,139), 4 )


        # Mark ground line
        XSTART = -5.
        XEND = 5.
        for xd in np.linspace( XSTART, XEND, 11 ):
            x = int(   (xd - XSTART) / (XEND - XSTART) * IM.shape[0]   )

            cv2.circle( IM, (x, 450), 5, (0,255,0), -1 )

            cv2.putText(IM, str(xd), (x-15,450+15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);


        # Draw Wheels of the cart
        wheel_1_pos = int(   (CART_POS - 1.2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        wheel_2_pos = int(   (CART_POS + 1.2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.circle( IM, (wheel_1_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_2_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_1_pos, 415), 2, (255,255,255), -1 )
        cv2.circle( IM, (wheel_2_pos, 415), 2, (255,255,255), -1 )

        # Cart base
        cart_base_start = int(   (CART_POS - 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        cart_base_end   = int(   (CART_POS + 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.line( IM, (cart_base_start, 380), (cart_base_end, 380), (255,255,255), 6 )

        # Pendulum hinge
        pendulum_hinge_x = int(   (CART_POS - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        pendulum_hinge_y = 380
        cv2.circle( IM, (pendulum_hinge_x, pendulum_hinge_y), 10, (255,255,255), -1 )


        # Pendulum
        pendulum_bob_x = int( LENGTH_OF_PENDULUM * np.cos( BOB_ANG / 180. * np.pi ) )
        pendulum_bob_y = int( LENGTH_OF_PENDULUM * np.sin( BOB_ANG / 180. * np.pi ) )
        cv2.circle( IM, (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), 10, (255,255,255), -1 )
        cv2.line( IM, (pendulum_hinge_x, pendulum_hinge_y), (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), (255,255,255), 3 )

        # Mark the current angle
        angle_display = BOB_ANG % 360
        if( angle_display > 180 ):
            angle_display = -360+angle_display
        cv2.putText(IM, "theta="+str( np.round(angle_display,4) )+" deg", (pendulum_hinge_x-15, pendulum_hinge_y-15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)


        # Display on top
        if t is not None:
            cv2.putText(IM, "t="+str(np.round(t,4))+"sec", (15, 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)
            cv2.putText(IM, "ANG="+str(np.round(BOB_ANG,4))+" degrees", (15, 35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)
            cv2.putText(IM, "POS="+str(np.round(CART_POS,4))+" m", (15, 55), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)



        return IM

In [144]:
sys = InvertedPendulum()

# Initial state
x0 = np.array([0, 0, np.pi/6, 0.])

# U = F @ x, loop and store U and x
x = x0
U = []
X = []

for i in range(100):
    u = F @ x
    x_dot = A @ x + B @ u
    
    x = x + x_dot * 0.01

    U.append(u)
    X.append(x)

In [145]:
# print eig values of A + BF
print("Eigen values of A + BF", np.linalg.eigvals(A + B @ F))

Eigen values of A + BF [ 0.          0.         -0.22592405  0.22592405]


In [147]:
syst = InvertedPendulum()

for i in range(len(X)):
    rendered = syst.step( [X[i][0], X[i][1], X[i][2], X[i][3]], t=i*0.01)
    print(X[i][2])
    cv2.imshow( 'im', rendered )

    # Press q to exit

    if cv2.waitKey(5) == ord('q'):
        break

0.5235987755982988
0.5236014481342156
0.5236067932060492
0.5236148108274407
0.5236255010256722
0.523638863841667
0.5236548993299898
0.5236736075588464
0.5236949886100848
0.5237190425791949
0.523745769575309
0.5237751697212029
0.5238072431532954
0.5238419900216499
0.5238794104899746
0.5239195047356232
0.5239622729495962
0.5240077153365412
0.5240558321147545
0.5241066235161815
0.5241600897864185
0.5242162311847132
0.5242750479839664
0.5243365404707332
0.5244007089452244
0.524467553721308
0.5245370751265107
0.5246092735020194
0.5246841492026829
0.5247617025970139
0.5248419340671906
0.5249248440090584
0.5250104328321321
0.5250987009595981
0.5251896488283161
0.5252832768888214
0.525379585605327
0.5254785754557265
0.5255802469315956
0.525684600538195
0.525791636794473
0.525901356233068
0.5260137594003109
0.5261288468562282
0.5262466191745443
0.526367076942685
0.5264902207617801
0.526616051246666
0.5267445690258895
0.5268757747417104
0.5270096690501052
0.5271462526207696
0.5272855261371229
0.